In [21]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

In [3]:
basedir = '/home/service/ipv6-measurements-mkoch/backup_2024/sra-analysis'
file_list = [f'{basedir}/data/scans/zmap_icmp_bgp_netaddr_1728140292_p0',
     f'{basedir}/data/bgp48_sra_checks/zmap_icmp_bgp_48_sra_1728075185_p0',
    f'{basedir}/data/bgp48_64s_sra_checks/zmap_icmp_bgp_48only_64s_sra_1730213648_processed',
    f'{basedir}/data/scans/zmap_icmp_route6_64_1728147418_p0',
    f'{basedir}/data/tuminput_sra_checks/zmap_icmp_tum_64_sra_1731694726_p0',
    ]
pickle_path = os.path.join(INTERIM_DATA_DIR, f"all_scans_joined_cached2.pkl")

In [4]:
df = cv.join_sra_scans(file_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:52<00:00, 46.59s/it]


In [5]:
df.head()

saddr,sumreplies,code,inscan,sumreplies_s1,code_s1,inscan_s1,sumreplies_s2,code_s2,inscan_s2,sumreplies_s3,code_s3,inscan_s3,sumreplies_s4,code_s4,inscan_s4
str,u32,str,bool,u32,str,bool,u32,str,bool,u32,str,bool,u32,str,bool
"""2402:b400:4440:53ef:c822:8bff:…",null,null,null,null,null,null,null,null,null,null,null,null,1,"""other""",true
"""2003:e6:37ff:3ab2:3ea6:2fff:fe…",null,null,null,null,null,null,null,null,null,null,null,null,1,"""other""",true
"""2405:8d40:4081:579:1807:c46a:e…",null,null,null,null,null,null,null,null,null,null,null,null,1,"""echoreply""",true
"""240e:358:9c0a:13ec:ae64:62ff:f…",null,null,null,null,null,null,null,null,null,null,null,null,1,"""other""",true
"""2408:8441:5808:10fa:1807:f6b5:…",null,null,null,null,null,null,null,null,null,null,null,null,1,"""other""",true


In [6]:
len(df)

132572621

In [8]:
df.select('saddr').write_csv(f'{INTERIM_DATA_DIR}/router-ips.csv')

In [9]:
df_tum = pl.read_csv(f'{basedir}/data/hitlists/responsive-addresses-20240921.txt',has_header=False,new_columns=['ipaddr'],comment_prefix='#')

In [11]:
df_tum.write_csv(f'{INTERIM_DATA_DIR}/hitlist-ips.csv')

In [16]:
itdk_dir = '/home/service/caida-itdk/ITDK-2024-08'

In [22]:
df_itdk01 = pl.read_csv(f'{itdk_dir}/speedtrap.nodes.addr',has_header=True)
df_itdk01 = df_itdk01.to_pandas()
df_itdk01['ipaddr_exploded'] = df_itdk01['ipaddr'].p_apply(lambda ip: ipaddress.IPv6Address(ip).exploded)
df_itdk01 = pl.from_pandas(df_itdk01)

<LAMBDA> DONE:   0%|          | 0/8954365 [00:00<?, ?it/s]

In [23]:
df_itdk02 = pl.read_csv(f'{itdk_dir}/speedtrap-snmp.nodes.addr',has_header=True).to_pandas()
df_itdk02['ipaddr_exploded'] = df_itdk02['ipaddr'].p_apply(lambda ip: ipaddress.IPv6Address(ip).exploded)
df_itdk02 = pl.from_pandas(df_itdk02)

<LAMBDA> DONE:   0%|          | 0/8949238 [00:00<?, ?it/s]

In [24]:
df_itdk03 = pl.read_csv(f'{itdk_dir}/itdk-run-20240829.addrs',has_header=False,new_columns=['ipaddr']).to_pandas()
df_itdk03['ipaddr_exploded'] = df_itdk03['ipaddr'].p_apply(lambda ip: ipaddress.IPv6Address(ip).exploded)
df_itdk03 = pl.from_pandas(df_itdk03)

<LAMBDA> DONE:   0%|          | 0/912865 [00:00<?, ?it/s]

In [25]:
df_itdk01 = df_itdk01.unique(subset=['ipaddr_exploded'])

In [26]:
df_itdk02 = df_itdk02.unique(subset=['ipaddr_exploded'])

In [27]:
df_itdk03 = df_itdk03.unique(subset=['ipaddr_exploded'])

In [28]:
df_itdk_joined = pl.concat([df_itdk01,df_itdk02,df_itdk03])
df_itdk_joined = df_itdk_joined.unique(subset=['ipaddr_exploded'])

In [29]:
df_itdk_joined = df_itdk_joined.filter(~pl.col('ipaddr').str.starts_with('ff00'))

In [30]:
df_itdk_joined

ipaddr,ipaddr_exploded
str,str
"""2a12:5240:1:5ae9:6e22:c616:a6a…","""2a12:5240:0001:5ae9:6e22:c616:…"
"""2600:9000:254a:d24d:a1f5:aa5f:…","""2600:9000:254a:d24d:a1f5:aa5f:…"
"""2600:9000:26bc:dd7c:162e:4b0e:…","""2600:9000:26bc:dd7c:162e:4b0e:…"
"""2a06:98c1:3108:2fa7:a331:e83b:…","""2a06:98c1:3108:2fa7:a331:e83b:…"
"""2001:268:92bb:98c3:0:1d:64a:a6…","""2001:0268:92bb:98c3:0000:001d:…"
…,…
"""2a05:4140:26aa:fa1f:c98b:85c:a…","""2a05:4140:26aa:fa1f:c98b:085c:…"
"""2a11:fb42:7db:a045:1902:7655:4…","""2a11:fb42:07db:a045:1902:7655:…"
"""2404:e801:1003::86b""","""2404:e801:1003:0000:0000:0000:…"


In [31]:
df_itdk_joined.select('ipaddr').write_csv(f'{INTERIM_DATA_DIR}/itdk-ips.csv')

In [32]:
ripe_dir = '/home/service/ripe-atlas-ipv6-traceroute'

In [33]:
df_topov6_1004 = pl.read_parquet(f'{ripe_dir}/ripe-topology-processed.parquet')

In [34]:
df_topov6_1004

ipaddr,is_src,is_dst,is_transit
str,i32,i32,i32
"""2a0f:9400:611f::1""",0,5,4
"""2a02:8071:51c2:71a1:e72:74ff:f…",0,0,108
"""2408:8957:1e30::1""",0,5,0
"""2001:610::1""",0,5,0
"""2001:fb1:fd0:107:b49b:2e16:d69…",0,0,3
…,…,…,…
"""2001:418:0:2000::55""",0,0,1
"""2a01:80a0:1000::1""",0,5,0
"""240e:2:8010:2:0:31:93:5203""",0,0,3


In [35]:
df_topov6_1004.select('ipaddr').write_csv(f'{INTERIM_DATA_DIR}/ripe-ips.csv')

In [36]:
ixp_dir = '/home/service/ixp-analysis/data'

In [37]:
df_ixp_src = pl.read_parquet(f'{ixp_dir}/sources_ixp')

In [38]:
df_ixp_src.head()

ipaddr,observed
str,i64
"""2a01:c22:914c:2200:1926:2619:3…",1
"""2a02:3031:20a:9010:81e2:2682:6…",1
"""2a02:8109:86a5:1a00:34d6:3066:…",1
"""2a00:20:604b:80ef:481b:1dec:9c…",1
"""2a02:2454:9bd8:200:5c25:6d0a:d…",2


In [39]:
df_ixp_dst = pl.read_parquet(f'{ixp_dir}/destinations_ixp')

In [40]:
df_ixp_all = pl.concat([df_ixp_src,df_ixp_dst])
df_ixp_all = df_ixp_all.group_by('ipaddr').agg(pl.col('observed').sum().alias('observed'))

In [42]:
df_ixp_all.select('ipaddr').write_csv(f'{INTERIM_DATA_DIR}/ixp-ips.csv')